In [1]:
from keras.applications import VGG16
import numpy as np

Using TensorFlow backend.


In [2]:
# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#include_top=False removes the output layer of the model
base_model = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [3]:
#To get only the name
base_model.layers[0].__class__.__name__
#To see the input/output of a particular layer
base_model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [4]:
#Freezing all the layers by making their trainable=False
for layer in base_model.layers:
    layer.trainable=False

In [5]:
#Checking this
base_model.layers[12].trainable

# Let's print our layers 
for (i,layer) in enumerate(base_model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [6]:
#See the ouput of the current model
base_model.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

In [7]:
#Now we add our dense layers for a new prediction on top of the base model
from keras.layers import Dense, Flatten
from keras.models import Sequential

top_model = base_model.output
top_model = Flatten()(top_model)
top_model = Dense(512, activation='relu')(top_model)   #First added FCL dense layer
top_model = Dense(512, activation='relu')(top_model)    #Second added FCL dense layer
top_model = Dense(256, activation='relu')(top_model)    #Third added FCL dense layer
top_model = Dense(5, activation='softmax')(top_model)    #Output layer with 5 class labels

In [8]:
#Now let's see the top_model output
top_model

<tf.Tensor 'dense_4/Softmax:0' shape=(None, 5) dtype=float32>

In [9]:
base_model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [10]:
# IMP: Mounting the base_model with the top_model
from keras.models import Model
newmodel = Model(inputs=base_model.input, outputs=top_model)

In [11]:
newmodel.output

<tf.Tensor 'dense_4/Softmax:0' shape=(None, 5) dtype=float32>

In [12]:
newmodel.layers

In [13]:
newmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [15]:
#Importing our images for recognition
from keras.preprocessing.image import ImageDataGenerator

train_data="D:/Users/OWNER/MLOps-ws/ImageRecogniser_TransferLearning/5 Celeb Dataset/data/train/"
test_data="D:/Users/OWNER/MLOps-ws/ImageRecogniser_TransferLearning/5 Celeb Dataset/data/val/"

#Data image augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data,
        target_size=(img_rows, img_cols),
        class_mode='categorical')

'''#Converting to 4D
for img_train in train_generator:
    img_train=np.expand_dims(img_train, axis=0)'''
 
test_generator = test_datagen.flow_from_directory(
        test_data,
        target_size=(img_rows, img_cols),
        class_mode='categorical',
        shuffle=False)

'''#Converting to 4D
for img_test in test_generator:
    img_test=np.expand_dims(img_test, axis=0)'''


Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


'#Converting to 4D\nfor img_test in test_generator:\n    img_test=np.expand_dims(img_test, axis=0)'

In [16]:
#Now let's compile our model
from keras.optimizers import RMSprop
newmodel.compile(optimizer = RMSprop(lr=0.0001),
                 loss = 'categorical_crossentropy',
                 metrics =['accuracy']
                )

In [17]:
history = newmodel.fit_generator(train_generator, epochs=3,steps_per_epoch=93, validation_data=test_generator)

Epoch 1/3
93/93 [==============================] - 1366s 15s/step - loss: 0.8074 - accuracy: 0.7055 - val_loss: 0.6634 - val_accuracy: 0.6800
Epoch 2/3
93/93 [==============================] - 5467s 59s/step - loss: 0.1585 - accuracy: 0.9601 - val_loss: 0.4293 - val_accuracy: 0.8800
Epoch 3/3
93/93 [==============================] - 825s 9s/step - loss: 0.0710 - accuracy: 0.9778 - val_loss: 0.3226 - val_accuracy: 0.8400


In [18]:
newmodel.save('5CelebRecognizer_VGG16.h5')

In [20]:
from keras.models import load_model

classifier = load_model('5CelebRecognizer_VGG16.h5')

In [22]:
#Testing the model

import os
import cv2
from os import listdir
from os.path import isfile, join

five_celeb_dict = {"[0]": "ben_afflek", 
                      "[1]": "elton_john",
                      "[2]": "jerry_seinfeld",
                      "[3]": "madonna",
                      "[4]": "mindy_kaling",
                     }

five_celeb_dict_n = {"ben_afflek": "ben_afflek", 
                      "elton_john": "elton_john",
                      "jerry_seinfeld": "jerry_seinfeld",
                      "madonna": "madonna",
                      "mindy_kaling": "mindy_kaling",
                      }

def draw_test(name, pred, im):
    celeb =five_celeb_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, celeb, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + five_celeb_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("D:/Users/OWNER/MLOps-ws/ImageRecogniser_TransferLearning/5 Celeb Dataset/val/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - ben_afflek
Class - madonna
Class - mindy_kaling
Class - madonna
Class - mindy_kaling
Class - mindy_kaling
Class - mindy_kaling
Class - ben_afflek
Class - madonna
Class - ben_afflek
